In [ ]:
PATH = "/home/chris/Documents/UNI_neu/Masterarbeit/other_data/100k_coursera_dataset/"
import pandas as pd
from os.path import join
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from tqdm import tqdm, tqdm_notebook
import numpy as np

from derive_conceptualspace.util.jsonloadstore import json_dump
from derive_conceptualspace.util.mpl_tools import show_hist

In [ ]:
df = pd.read_csv(join(PATH, "reviews_by_course.csv"))
print(len(df))
print(len(df["CourseId"].unique()))
df.head()

In [ ]:
#df["Review"].str.len().hist(bins=1000, log=True)
lens = [i if not np.isnan(i) else 0 for i in df["Review"].str.len()]
show_hist(lens, "Lens per Review", xlabel="Lengths in chars", ylabel="Count", cutoff_percentile=95, log=False)

## Filtering (only english, long enough, ..)

In [ ]:
lans = []
for item in tqdm_notebook(df["Review"]):
    if not isinstance(item, str) or len(item) < 40:
        lans.append("tooshort")
    else:
        try:
            lans.append(detect(item))
        except LangDetectException as e:
            lans.append("unk")
df["lan"] = lans

In [ ]:
ens = df[df["lan"] == "en"]
grouped_reviews = ens.groupby("CourseId")['Review'].apply(lambda x: '. \n'.join(x))

In [ ]:
show_hist(grouped_reviews.str.len(), "Lens per All-Reviews-Per-Course", xlabel="Lengths in chars", ylabel="Count", cutoff_percentile=92, log=False)

In [ ]:
show_hist([i for i in grouped_reviews.str.len() if i > 2000], "Lens per All-Reviews-Per-Course", xlabel="Lengths in chars", ylabel="Count", cutoff_percentile=92, log=False)

In [ ]:
tmp = grouped_reviews[grouped_reviews.str.len() > 2000]
print(tmp)
#for k,v in dict(tmp[:10]).items():
#    print("\n\n"+k)
#    print("  "+("\n  ".join(v.split("\n"))))

In [ ]:
json_dump(dict(tmp), "/home/chris/Documents/UNI_neu/Masterarbeit/other_data/100k_coursera_dataset/usable_reviews.json")

In [ ]:
tmp.to_csv("/home/chris/Documents/UNI_neu/Masterarbeit/other_data/100k_coursera_dataset/usable_reviews.csv")

In [ ]:
json_dump(dict(langs={i: "en" for i in tmp.index}), "/home/chris/Documents/UNI_neu/Masterarbeit/other_data/100k_coursera_dataset/languages.json")